In [1]:
!pip install medspacy
!python -m spacy download en_core_web_sm
!pip install target_matcher

2024-03-19 08:39:36.012748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 08:39:36.012809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 08:39:36.014394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 08:39:36.023056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 08:39:37.221901: W tensorflow/compiler/tf2

In [2]:
import medspacy
from medspacy.ner import TargetRule

nlp = medspacy.load()

# Correctly access the target matcher component
target_matcher = nlp.get_pipe("medspacy_target_matcher")

# Define target rules for anatomy, radiology findings, and specifically for PI-RADS scores
target_rules = [
    TargetRule(literal="left peripheral zone", category="ANATOMY"),
    TargetRule(literal="right peripheral zone", category="ANATOMY"),
    TargetRule(literal="anterior fibromuscular stroma", category="ANATOMY"),
    TargetRule(literal="significant restricted diffusion", category="RADIOLOGY_FINDING"),
    TargetRule(literal="focal early enhancement", category="RADIOLOGY_FINDING"),
    # Define a pattern to capture "PI-RADS Score" followed by any digit
    TargetRule(literal="PI-RADS Score", category="RADIOLOGY_FINDING", pattern=[{"LOWER": {"REGEX": "pi-rads"}}, {"LOWER": "score"}, {"LOWER": ":"}, {"IS_DIGIT": True}]),
]

# Add these rules to the target matcher
for rule in target_rules:
    target_matcher.add([rule])

# Example text
text = """
Patient Information

Patient Name: Mr. Smith
Patient ID: 789456
Date of Birth: 06/25/1965
Date of Examination: 03/15/2024

MRI Examination Details

MRI Technique: Multi-parametric MRI with T2-weighted images, diffusion-weighted imaging (DWI), and dynamic contrast-enhanced (DCE) MRI.
Prostate Volume: 45 cc
PSA Level: 5.8 ng/mL

MRI Findings

Lesion 1: Located in the left peripheral zone, mid gland. It shows significant restricted diffusion with a corresponding low ADC value and focal early enhancement on DCE MRI. PI-RADS Score: 5.
Lesion 2: Located in the right peripheral zone, apex. It has mildly restricted diffusion and no significant enhancement on DCE MRI. PI-RADS Score: 3.
Lesion 3: Located in the anterior fibromuscular stroma, showing moderate restricted diffusion and no early enhancement. PI-RADS Score: 3.
"""

# Process the text
doc = nlp(text)

# Print extracted entities
for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

# Specifically, print PI-RADS Scores
print("\nPI-RADS Scores:")
pi_rads_scores = [ent for ent in doc.ents if ent.label_ == "PI-RADS_SCORE"]
for ent in pi_rads_scores:
    print(ent.text)


left peripheral zone: ANATOMY
significant restricted diffusion: RADIOLOGY_FINDING
focal early enhancement: RADIOLOGY_FINDING
right peripheral zone: ANATOMY
anterior fibromuscular stroma: ANATOMY

PI-RADS Scores:


In [3]:
from spacy import displacy

# Define colors for different entity types
colors = {
    "ANATOMY": "lightgreen",
    "RADIOLOGY_FINDING": "lightblue"
}

options = {
    "ents": ["ANATOMY", "RADIOLOGY_FINDING"],
    "colors": colors
}

# Render the entities in the doc with custom colors
displacy.render(doc, style="ent", jupyter=True, options=options)
